<center>
<hr>
<h1>INF-477. Redes Neuronales Artificiales.</h1>
<h2>Tarea 2 - Autoencoders, RBMs y ConvNets</h2>
<hr>
</center>

<div style="width:25%; display: inline-block"></div>
<div style="width:25%; display: inline-block">
    <b>Juan Carlos Garcés Bernt</b><br>
    jcgarces@alumnos.inf.utfsm.cl
</div>
<div style="width:25%; display: inline-block;">
    <b>Natalia Gonzalez</b><br>
    natalia.gonzalezg@usm.cl
</div>
<div style="width:25%; display: inline-block"></div>


<h1>2 Aprendizaje Semi.Supervisado en NORB</h1>

Como hemos discutido en clases, uno de los problemas más relevantes a la hora de aplicar técnicas de aprendizaje
automático a problemas es reales es el requisito de disponer de un gran número de datos etiquetados,
es decir, ejemplos para los que se conoce la respuesta deseada del sistema. Un problema de aprendizaje para
el que existen pocos datos etiquetados y muchos datos no etiquetados se denomina *semi-supervisado*. En
esta sección, utilizaremos la idea de pre-entrenar una red en modo no supervisado para atacar problemas
de aprendizaje semi-supervisado. Con este objetivo en mente, trabajaremos con un dataset denominado
$NORB$, introducido en [9] y utilizado en [10], que corresponde a imágenes estéreo de juguetes clasificados en
$6$ categorías. Se tienen $291.600$ ejemplos de entrenamiento y $58.320$ ejemplos de pruebas.
<img src="img/Fig2.png">
<center>Fig. 2: Dataset NORB.</center>
Los datos asociados a esta actividad podrán ser obtenidos utilizando los siguientes comandos en la línea
de comandos (sistemas UNIX)
```
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_1
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_2
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_3
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_4
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_5
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_6
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_7
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_8
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_9
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_10
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_11
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_12
wget http://octopus.inf.utfsm.cl/~ricky/data_batch_13
```
Los primeros $10$ batches corresponden a los datos de entrenamiento y los últimos $2$ a los datos de pruebas. Los
archivos corresponden a diccionarios serializados de python y pueden ser "extraídos" utilizando la siguiente
función

In [8]:
import cPickle
import scipy
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils 
from numpy.random import binomial

from sklearn.neural_network import BernoulliRBM
from keras.models import load_model


In [2]:
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def codifica(Y):
    for i in range(0,len(Y)):
        Y[i] = np.array([Y[i]])
    return np.array(Y)

Una vez extraído, cada diccionario contendrá $2$ elementos importantes: *data* y *labels*. El primer elemento
(*data*) es un matriz de $2048 \times n$ (numpy array). Cada columna de esa matriz corresponde a una imagen
estéreo de un juguete: los primeros $1024$ valores vienen de una de las cámaras/vistas y los siguientes $1024$ de
la otra. Por otro lado, el elemento (*labels*) del diccionario contiene una lista de $n$ valores enteros entre $0$ y $5$ que identifican las clases antes a las que pertenecen los juguetes.

a) Construya una función que cargue todos los bloques de entrenamiento y pruebas del problema NORB
generando como salida: <br>
(i) dos matrices $X_{tr}, Y_{tr}$, correspondientes a las imágenes y etiquetas de entrenamiento, <br>
(ii) dos matrices $X_t, Y_t$, correspondientes a las imágenes y etiquetas de pruebas, y finalmente<br>
(iii) dos matrices $X_v, Y_v$, correspondientes a imágenes y etiquetas que se usarán como conjunto de
validación, es decir para tomar decisiones de diseño acerca del modelo. Este último conjunto debe ser
extraído desde el conjunto de entrenamiento seleccionando $5.832$ casos de cada batch.

In [ ]:
def load_NORB_train(PATH):
    xtr = []
    ytr = []
    
    for b in range(1,11):
        
        f = os.path.join(PATH,'data_batch_%d' % (b, ))
        datadict = unpickle(f)
        X = datadict['data'].T
        Y = datadict['labels']
        Y = codifica(Y)
        Z = np.concatenate((X,Y),axis=1)
        del X,Y
        np.random.shuffle(Z)
        xtr.append(Z[5832:,0:-1])
        ytr.append(Z[5832:,-1])

        del Z
      
    Xtr = np.concatenate(xtr)
    del xtr
    Ytr = np.concatenate(ytr)
    Y_tr = np_utils.to_categorical(Ytr, 6)
    del ytr,Ytr
    return Xtr, Y_tr
    
def load_NORB_val(PATH):
    xval = []
    yval = []
    
    for b in range(1,11):
        
        f = os.path.join(PATH,'data_batch_%d' % (b, ))
        datadict = unpickle(f)
        X = datadict['data'].T
        Y = datadict['labels']
        Y = codifica(Y)
        Z = np.concatenate((X,Y),axis=1)
        del X,Y
        np.random.shuffle(Z)
        xval.append(Z[:5832,0:-1])
        yval.append(Z[:5832,-1])
        del Z
      
    Xval = np.concatenate(xval)
    del xval
    Yval = np.concatenate(yval)
    Y_val = np_utils.to_categorical(Yval, 6)
    del yval,Yval

    return Xval, Y_val

In [3]:
def load_NORB_test(PATH):
    xts = []
    yts = []
    
    for b in range(11,13):
        f = os.path.join(PATH,'data_batch_%d' % (b, ))
        datadict = unpickle(f)
        X = datadict['data'].T
        Y = datadict['labels']
        Y = codifica(Y)
        Z = np.concatenate((X,Y),axis=1)
        del X,Y
        np.random.shuffle(Z)
        xts.append(Z[:,0:-1])
        yts.append(Z[:,-1])
        del Z
    
    Xts = np.concatenate(xts)
    del xts
    Yts = np.concatenate(yts)
    Y_ts = np_utils.to_categorical(Yts, 6)
    del yts,Yts
    return Xts, Y_ts

b) Construya una función que escale apropiadamente las imágenes antes de trabajar. Experimente escalando
linealmente los datos de tal forma que cada pixel quede en el intervalo $[-1,1]$ con el máximo y
mínimo valor observado en los extremos del intervalo. Evalúe más tarde la ventaja de centrar y escalar
los datos para que cada atributo (pixel) tenga desviación estándar $1$ y media nula.

In [ ]:
# Escalamiento lineal con valores en [-1,1]
def rescale_img(X):
    scaler = MinMaxScaler(feature_range=(-1,1))
    X_scaled = scaler.fit_transform(X)
    return X_scaled

# Centrado y escalado con desviacion estandar 1 y media 0
def stand_img(X):
    scaler = StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    return X_scaled


c) Su objetivo será ahora evaluar el desempeño de una red FF en un escenario semi-supervisado. Para
ello simulará un situación en la que se tienen $n_s$ ejemplos de entrenamiento para los cuales se conoce
la etiqueta correcta y $n_{ns} = n_{tr} - n_s$ ejemplos para los cuales no se tiene esta información ($n_{tr}$ es el
número total de ejemplos de entrenamiento). Para empezar, deberá entrenar una red FF con salida
softmax para el problema NORB. Considere la inclusión de dos capas escondidas (de $4000$ y $2000$
unidades) y funciones de activación $relu$. Como parámetros de referencia considere: BP con tasa de
aprendizaje constante, función de pérdida *cross-entropy binaria*, y mini batches de tamaño $10$. Puede
utilizar el conjunto de validación para mejorar el entrenamiento. 

In [ ]:
def pre_train_AE(Xtr,Xval,n_hidden_layer1,n_hidden_layer2):
    #AE1
    input_img1 = Input(shape=(2048,))
    encoded1 = Dense(n_hidden_layer1,activation='relu')(input_img1)
    decoded1 = Dense(2048, activation='relu')(encoded1)
    autoencoder1 = Model(input=input_img1, output=decoded1)
    encoder1 = Model(input=input_img1, output=encoded1)
    autoencoder1.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder1.fit(Xtr, Xtr, nb_epoch=50, batch_size=20,shuffle=True, validation_data=(Xval, Xval))
    encoded_input1 = Input(shape=(n_hidden_layer1,))

    #AE2
    x_train_encoded1 = encoder1.predict(Xtr)
    x_val_encoded1 = encoder1.predict(Xval)
    input_img2 = Input(shape=(n_hidden_layer1,))
    encoded2 = Dense(n_hidden_layer2, activation='relu')(input_img2)
    decoded2 = Dense(n_hidden_layer1, activation='relu')(encoded2)
    autoencoder2 = Model(input=input_img2, output=decoded2)
    encoder2 = Model(input=input_img2, output=encoded2)
    autoencoder2.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder2.fit(x_train_encoded1,x_train_encoded1,nb_epoch=50,batch_size=20,shuffle=True, validation_data=(x_val_encoded1, x_val_encoded1))
    encoded_input2 = Input(shape=(n_hidden_layer2,))
    
    w = []
    w.append(autoencoder1.layers[1].get_weights())
    w.append(autoencoder2.layers[1].get_weights())
    return w
    


def pre_train_dAE(x_train,x_val,n_hidden_layer1,n_hidden_layer2):
    ### NOISE 
    noise_level = 0.1
    noise_mask = binomial(n=1,p=noise_level,size=x_train.shape)
    noisy_x_train = x_train*noise_mask
    noise_mask = binomial(n=1,p=noise_level,size=x_val.shape)
    noisy_x_val = x_val*noise_mask

    ###d_AUTOENCODER 1
    input_img1 = Input(shape=(2048,))
    encoded1 = Dense(n_hidden_layer1,activation='relu')(input_img1)
    decoded1 = Dense(2048, activation='relu')(encoded1)
    autoencoder1 = Model(input=input_img1, output=decoded1)
    encoder1 = Model(input=input_img1, output=encoded1)
    autoencoder1.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder1.fit(noisy_x_train, x_train, nb_epoch=50, batch_size=20,shuffle=True, validation_data=(noisy_x_val, x_val))
    encoded_input1 = Input(shape=(n_hidden_layer1,))
    
    ###d_AUTOENCODER 2
    x_train_encoded1 = encoder1.predict(noisy_x_train) 
    x_val_encoded1 = encoder1.predict(noisy_x_val)
    input_img2 = Input(shape=(n_hidden_layer1,))
    encoded2 = Dense(n_hidden_layer2, activation='relu')(input_img2)
    decoded2 = Dense(n_hidden_layer1, activation='relu')(encoded2)
    autoencoder2 = Model(input=input_img2, output=decoded2)
    encoder2 = Model(input=input_img2, output=encoded2)
    autoencoder2.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy')
    autoencoder2.fit(x_train_encoded1,x_train_encoded1,nb_epoch=50,batch_size=20,shuffle=True, validation_data=(x_val_encoded1, x_val_encoded1))
    encoded_input2 = Input(shape=(n_hidden_layer2,))

    w = []
    w.append(autoencoder1.layers[1].get_weights())
    w.append(autoencoder2.layers[1].get_weights())
    return w
    

def pre_train_RBM(x_train,n_hidden_layer1,n_hidden_layer2):
    
    rbm1 = BernoulliRBM(n_components=n_hidden_layer1, batch_size=25,learning_rate=0.05,verbose=0, n_iter=50)
    rbm1.fit(x_train)
    encoded_train1 = rbm1.transform(x_train)
    weights1 = []
    weights1.append(rbm1.components_.T)
    weights1.append(rbm1.intercept_hidden_.T)
    
    rbm2 = BernoulliRBM(n_components=n_hidden_layer2, batch_size=25,learning_rate=0.05,verbose=0, n_iter=50)
    rbm2.fit(encoded_train1)
    weights2 = []
    weights2.append(rbm2.components_.T)
    weights2.append(rbm1.intercept_hidden_.T)
    
    w = []
    w.append(weights1)
    w.append(weights2)
    return w
    
    

def ann_ff(Xtr,Ytr,Xval,Yval,Xts,Yts,scale=0,theta=1,pre_train='None',act_func='relu'):
    n_hidden_layer1 = 4000
    activation_layer1 = act_func
    n_hidden_layer2 = 2000
    activation_layer2 = act_func
    optimizer_ = SGD(lr=1.0)
    n_ns = int((1.0-theta)*len(Ytr))
    
    #########
    # ESCALAMIENTO DE LAS IMAGENES
    # scale = 0, no se escalan las imagenes
    # scale = 1, se aplica escalamiento lineal
    # scale = 2, se aplica estandarizacion de las imagenes (desv. estandar 1 y media nula)
    escala = 'None'
    if scale !=0:
        if(scale == 1):
            rescale_img(Xtr)
            rescale_img(Xval)
            rescale_img(Xts)
            escala = 'Linear'
        elif(scale == 2):
            stand_img(Xtr)
            stand_img(Xval)
            stand_img(Xts)
            escala = 'Standar Scale'
        else:
            return -1
    ########
    
    ##########
    ### PRE-ENTRENAMIENTO
    if pre_train != 'None':
        if pre_train == 'AE':
            PreTrained = pre_train_AE(Xtr[:n_ns,:],Xval,n_hidden_layer1,n_hidden_layer2) 
        elif pre_train == 'dAE':
            PreTrained = pre_train_dAE(Xtr[:n_ns,:],Xval,n_hidden_layer1,n_hidden_layer2)
        elif pre_train == 'RBM':
            PreTrained = pre_train_RBM(Xtr[:n_ns,:],n_hidden_layer1,n_hidden_layer2)
        else:
            return -1
    ###########

    model = Sequential()
    model.add(Dense(n_hidden_layer1, activation=activation_layer1,input_shape=(2048,)))
    if pre_train != 'None':
        model.layers[-1].set_weights(PreTrained[0])
    model.add(Dense(n_hidden_layer2, activation=activation_layer2))
    if pre_train != 'None':
        model.layers[-1].set_weights(PreTrained[1])

    model.add(Dense(6, activation='softmax'))
    model.summary()  
    
    model.compile(optimizer=optimizer_,loss='binary_crossentropy', metrics=['accuracy'])
        
    
    if pre_train != 'None':
    model.fit(Xtr[n_ns:,:], Ytr[n_ns:], nb_epoch=0, batch_size=10,shuffle=True, validation_data=(Xval, Yval),verbose=1)
    
    else:
        ################
        #####Entrenamiento SemiSupervisado
        n_ns = int((1.0-theta)*len(Ytr))
        n_s = len(Ytr)- n_ns
        N_NS = n_ns/32
        N_S = n_s/32
        print "n_ns:",n_ns," n_s:",n_s," NS:", N_S," NNS:",N_NS
        if n_ns == 0:
            model.fit(Xtr, Ytr, nb_epoch=50, batch_size=20,shuffle=True, validation_data=(Xval, Yval),verbose=1)
        else:
            for i in range(0,32):
                print "fit: [",N_NS*i+N_S*i,":",N_NS*(i)+N_S*(i+1),"]"
                model.fit(Xtr[N_NS*i+N_S*i:N_NS*(i)+N_S*(i+1),:], Ytr[N_NS*i+N_S*i:N_NS*(i)+N_S*(i+1)], nb_epoch=15, batch_size=10,shuffle=True, validation_data=(Xval[:N_S,:], Yval[:N_S]),verbose=1)
                Y_pred = np.array([])
                print "pred: [",N_NS*(i)+N_S*(i+1),":",N_NS*(i+1)+N_S*(i+1),"]"
                Y_pred = model.predict(Xtr[N_NS*(i)+N_S*(i+1):N_NS*(i+1)+N_S*(i+1),:])
                model.fit(Xtr[N_NS*(i)+N_S*(i+1):N_NS*(i+1)+N_S*(i+1),:], Y_pred, nb_epoch=15, batch_size=10,shuffle=True, validation_data=(Xval[:N_S,:], Yval[:N_S]),verbose=1)
        ################
    
    model.save(act_func+'Net-'+escala+'_scale-'+str(theta)+'_theta-'+pre_train+'_pretrain.h5')

In [5]:
def evaluador(X,Y,scale=0,theta=1,pre_train='None',act_func='relu'):
    escala = 'None'
    if(scale == 1): 
        escala = 'Linear'
    elif(scale == 2): 
        escala = 'Standar Scale'
    
    ann = act_func+'Net-'+escala+'_scale-'+str(theta)+'_theta-'+pre_train+'_pretrain.h5'
    model = load_model(ann)
    score = model.evaluate(X, Y, verbose=0)
    print "Theta:",theta," ---- Scale:", escala
    print "Activation function:", act_func," ---- Pre-Train:", pre_train
    print "Test loss:", score[0]
    print "Test accuracy:", score[1]
    return score

In [ ]:
def graficador(A,B,titulo='_',y_label='_'):
    plt.plot(A,B)
    plt.title(titulo)
    plt.xlabel('theta')
    plt.ylabel(y_label)
    plt.show()

Xtr, Ytr = load_NORB_train(".")
Xval, Yval = load_NORB_val(".")

Xts, Yts = load_NORB_test(".")

Theta = np.linspace(0,1,num=11)
PT = ['None', 'AE', 'dAE', 'RBM'] 
AF = ['relu', 'tanh', 'sigmoid']

#### Generando las ff ######
for act_func_ in AF:
    for pre_train_ in PT:
        for scale_ in range(0,3):
            for theta_ in Theta:
                ann_ff(Xtr,Ytr,Xval,Yval,Xts,Yts,scale=scale_,theta=theta_,pre_train=pre_train_,act_func=act_func_)

In [9]:
Xts, Yts = load_NORB_test(".")
evaluador(Xts,Yts,scale=0,theta=1,pre_train='None',act_func='relu')

Theta: 1  ---- Scale: None
Activation function: relu  ---- Pre-Train: None
Test loss: 4.45284479596
Test accuracy: 0.722222221797


[4.4528447959648725, 0.72222222179705875]

Construya un gráfico que muestre
cómo evoluciona el error de pruebas como función de $\theta_s = n_s/n_{tr}$. Experimente con $\theta_s = 0.1, 0.2,...1$.

In [ ]:
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='None',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Entrenamiento SemiSupervisado: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Entrenamiento SemiSupervisado: theta v/s accuracy',y_label='accuracy')


d) Su objetivo será ahora construir un gráfico similar al anterior que muestre cómo evoluciona el error de pruebas como función de $\theta_s = n_s / n_{tr}$ cuando la red se pre-entrena utilizando los datos no supervisados. ¿Mejora el resultado con respecto a la red entrenada utilizando sólo los casos para los que se conoce la etiqueta? Experimente pre-entrenando con distintas estrategias ( por ejemplo AE's versus dAe's ó AE's versus RBM's).

### Todos etiquetados y sin pre-entrenamiento

In [ ]:
for esc in range(0,3):
    score = evaluador(Xts,Yts,scale=esc,theta=1,pre_train='None',act_func=funcion)
    print "loss=",score[0]
    print "accuracy=",score[1]

### Variando theta y con pre-entrenamiento AE

In [ ]:
for esc in range(0,3):	
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='AE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento AE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento AE: theta v/s accuracy',y_label='accuracy')

### Variando theta y con pre-entrenamiento dAE

In [ ]:
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='dAE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento dAE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento dAE: theta v/s accuracy',y_label='accuracy')

### Variando theta y con pre-entrenamiento RBM

In [ ]:
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='RBM',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento RBM: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento RBM: theta v/s accuracy',y_label='accuracy')

e) Repita el experimento anterior cambiando las funciones de activación a $sigmoidales$ y $tanh$.

### Función de Activación Sigmoid

In [ ]:
funcion = 'sigmoid'
#c)
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='None',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Entrenamiento SemiSupervisado: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Entrenamiento SemiSupervisado: theta v/s accuracy',y_label='accuracy')
    

In [ ]:
#d)
# Todos etiquetados y sin pre-entrenamiento
for esc in range(0,3):
    score = evaluador(Xts,Yts,scale=esc,theta=1,pre_train='None',act_func=funcion)
    print "loss=",score[0]
    print "accuracy=",score[1]

In [ ]:
# Variando theta y con pre-entrenamiento AE
for esc in range(0,3):	
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='AE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento AE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento AE: theta v/s accuracy',y_label='accuracy')

In [ ]:
# Variando theta y con pre-entrenamiento dAE
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='dAE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento dAE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento dAE: theta v/s accuracy',y_label='accuracy')

In [ ]:
# Variando theta y con pre-entrenamiento RBM
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='RBM',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento RBM: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento RBM: theta v/s accuracy',y_label='accuracy')

### Función de Activación Tanh

In [ ]:
funcion = 'tanh'
#c)
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='None',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Entrenamiento SemiSupervisado: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Entrenamiento SemiSupervisado: theta v/s accuracy',y_label='accuracy')

In [ ]:
#d)
# Todos etiquetados y sin pre-entrenamiento
for esc in range(0,3):
    score = evaluador(Xts,Yts,scale=esc,theta=1,pre_train='None',act_func=funcion)
    print "loss=",score[0]
    print "accuracy=",score[1]

In [ ]:
# Variando theta y con pre-entrenamiento AE
for esc in range(0,3):	
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='AE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento AE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento AE: theta v/s accuracy',y_label='accuracy')

In [ ]:
# Variando theta y con pre-entrenamiento dAE
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='dAE',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento dAE: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento dAE: theta v/s accuracy',y_label='accuracy')

In [ ]:
# Variando theta y con pre-entrenamiento RBM
for esc in range(0,3):
    loss_ = []
    accuracy_ = []
    for i in range(1,11):
        score = evaluador(Xts,Yts,scale=esc,theta=Theta[i],pre_train='RBM',act_func=funcion)
        loss_.append(score[0])
        accuracy_.append(score[1])

    graficador(Theta,loss_,titulo='Pre-entrenamiento RBM: theta v/s loss',y_label='loss')
    graficador(Theta,accuracy_,titulo='Pre-entrenamiento RBM: theta v/s accuracy',y_label='accuracy')